# Introduction

In this project, we are going to apply the K-Means clustering to solve for the following problems:

- How has the price of rice changed in different regions over time? 
- Which regions experience the most volatile rice prices? 
- Is there an existing seasonal pattern in rice price volatility? 

In [207]:
import pandas as pd

pd.set_option('display.max_columns', 500)
food_prices_data = pd.read_csv("wfp_food_prices_phl.csv", skiprows=[1])

# Data Cleaning and Exploration

In [208]:
# Number of data and features 
print(food_prices_data.shape)
print(len(str(food_prices_data.shape))*'-')

# Check how many types of data are in the dataset
print(food_prices_data.dtypes.value_counts())
print(len(str(food_prices_data.shape))*'-')

# Check the first 16 columns
food_prices_data.head(16)

(121512, 14)
------------
object     10
float64     4
Name: count, dtype: int64
------------


,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
0,2000-01-15,National Capital region,Metropolitan Manila,Metro Manila,14.604167,120.982222,cereals and tubers,Maize flour (yellow),KG,actual,Retail,PHP,15.00,0.3717
1,2000-01-15,National Capital region,Metropolitan Manila,Metro Manila,14.604167,120.982222,cereals and tubers,"Rice (milled, superior)",KG,actual,Retail,PHP,20.00,0.4957
2,2000-01-15,National Capital region,Metropolitan Manila,Metro Manila,14.604167,120.982222,cereals and tubers,"Rice (milled, superior)",KG,actual,Wholesale,PHP,18.35,0.4548
3,2000-01-15,National Capital region,Metropolitan Manila,Metro Manila,14.604167,120.982222,cereals and tubers,"Rice (regular, milled)",KG,actual,Wholesale,PHP,16.35,0.4052
4,2000-01-15,Region III,Nueva Ecija,Palayan,15.541500,121.084800,cereals and tubers,"Rice (milled, superior)",KG,actual,Retail,PHP,19.00,0.4709
5,2000-01-15,Region III,Nueva Ecija,Palayan,15.541500,121.084800,cereals and tubers,"Rice (milled, superior)",KG,actual,Wholesale,PHP,18.00,0.4461
6,2000-01-15,Region III,Nueva Ecija,Palayan,15.541500,121.084800,cereals and tubers,"Rice (regular, milled)",KG,actual,Retail,PHP,18.10,0.4486
7,2000-01-15,Region IX,Zamboanga del Sur,Zamboanga City,6.910255,122.071715,cereals and tubers,"Rice (regular, milled)",KG,actual,Retail,PHP,16.90,0.4188
8,2000-01-15,Region VI,Iloilo,Iloilo City,10.696944,122.564444,cereals and tubers,"Rice (milled, superior)",KG,actual,Retail,PHP,20.40,0.5056
9,2000-01-15,Region VI,Iloilo,Iloilo City,10.696944,122.564444,cereals and tubers,"Rice (milled, superior)",KG,actual,Wholesale,PHP,15.60,0.3866


### Check for Missing Values

In [209]:
food_prices_data.isnull().sum()

date         0
admin1       0
admin2       0
market       0
latitude     0
longitude    0
category     0
commodity    0
unit         0
priceflag    0
pricetype    0
currency     0
price        0
usdprice     0
dtype: int64

### Remove Unnecessary Data

In [210]:
food_prices_data = food_prices_data.drop(columns=["category", "currency"])

food_prices_data.head()

,date,admin1,admin2,market,latitude,longitude,commodity,unit,priceflag,pricetype,price,usdprice
0,2000-01-15,National Capital region,Metropolitan Manila,Metro Manila,14.604167,120.982222,Maize flour (yellow),KG,actual,Retail,15.00,0.3717
1,2000-01-15,National Capital region,Metropolitan Manila,Metro Manila,14.604167,120.982222,"Rice (milled, superior)",KG,actual,Retail,20.00,0.4957
2,2000-01-15,National Capital region,Metropolitan Manila,Metro Manila,14.604167,120.982222,"Rice (milled, superior)",KG,actual,Wholesale,18.35,0.4548
3,2000-01-15,National Capital region,Metropolitan Manila,Metro Manila,14.604167,120.982222,"Rice (regular, milled)",KG,actual,Wholesale,16.35,0.4052
4,2000-01-15,Region III,Nueva Ecija,Palayan,15.541500,121.084800,"Rice (milled, superior)",KG,actual,Retail,19.00,0.4709


Since we will only be focusing on rice prices, the category column is unnecessary because we will only have 1 category. Currency will also be removed since all values in price column are in PH 

In [211]:
food_prices_data = food_prices_data.rename(columns={"price": "price (PH)"})


### Check Column Values

In [212]:
food_prices_data["admin1"].value_counts()

admin1
Region III                              10697
Region VI                                9110
Region V                                 9002
Region VIII                              8510
Region XI                                8127
Cordillera Administrative region         7688
Region IV-A                              7253
Region X                                 7226
Region IV-B                              7118
Region XIII                              6959
Region XII                               6955
Region VII                               6731
Region II                                5887
Region I                                 5860
Region IX                                5837
Autonomous region in Muslim Mindanao     5613
National Capital region                  2939
Name: count, dtype: int64

In [213]:
food_prices_data["admin2"].value_counts()

admin2
Davao del Sur        4719
Zamboanga del Sur    3416
Cebu                 3381
Iloilo               3136
South Cotabato       3066
                     ... 
Dinagat Islands       878
Sulu                  843
Tawi-Tawi             698
Lanao del Sur         644
Camarines Sur         158
Name: count, Length: 79, dtype: int64

In [214]:
food_prices_data["market"].value_counts()

market
Davao City             3272
Metro Manila           2939
Cebu City              2087
Iloilo City            1946
Zamboanga City         1880
                       ... 
Butuan City             265
Cagayan de Oro City     253
Davao Occidental        196
Naga City               158
Calapan City            146
Name: count, Length: 108, dtype: int64

In [215]:
food_prices_data["pricetype"].value_counts()

pricetype
Retail       116481
Wholesale      4367
Farm Gate       664
Name: count, dtype: int64

In [216]:
commodity_counts = food_prices_data["commodity"].value_counts().reset_index()
commodity_counts.columns = ["commodity", "count"]
print(commodity_counts.to_string(index=False))


                          commodity  count
             Rice (regular, milled)   4969
                            Cabbage   4459
                          Eggplants   3528
                 Rice (well milled)   3518
                    Fish (milkfish)   3389
                       Bitter melon   3375
                   Fish (roundscad)   3272
                     Rice (special)   3235
                            Coconut   3186
                       Beans (mung)   3171
            Meat (pork, with bones)   3170
                     Fish (tilapia)   3074
                  Bananas (lakatan)   2876
                           Squashes   2875
            Rice (milled, superior)   2850
                          Anchovies   2764
                              Choko   2630
                             Ginger   2523
                           Tomatoes   2507
                               Eggs   2473
                 Bananas (latundan)   2468
                  Mangoes (carabao)   2412
           

In [217]:
food_prices_data["priceflag"].value_counts()

priceflag
actual       117396
aggregate      4116
Name: count, dtype: int64

### Change Data Types

Convert price, latitude, and longitude to numbers, and date to datetime format. Invalid entries are safely turned into NaN or NaT using errors="coerce". 

In [218]:
food_prices_data["price (PH)"] = pd.to_numeric(food_prices_data["price (PH)"], errors="coerce")
food_prices_data["latitude"] = pd.to_numeric(food_prices_data["latitude"], errors="coerce")
food_prices_data["longitude"] = pd.to_numeric(food_prices_data["longitude"], errors="coerce")
food_prices_data["date"] = pd.to_datetime(food_prices_data["date"], errors="coerce")

print(food_prices_data.dtypes.value_counts())

object            7
float64           4
datetime64[ns]    1
Name: count, dtype: int64


### Filter Rice From Commodities

In [219]:
all_rice_df = food_prices_data[food_prices_data["commodity"].str.startswith("Rice", na=False)]

all_rice_df["commodity"].value_counts()



commodity
Rice (regular, milled)     4969
Rice (well milled)         3518
Rice (special)             3235
Rice (milled, superior)    2850
Rice (paddy)                664
Rice (premium)              598
Name: count, dtype: int64

Since we only need rice commodities, we need to filter out all data that aren't rice

In [220]:
all_rice_df.head(16)

,date,admin1,admin2,market,latitude,longitude,commodity,unit,priceflag,pricetype,price (PH),usdprice
1,2000-01-15,National Capital region,Metropolitan Manila,Metro Manila,14.604167,120.982222,"Rice (milled, superior)",KG,actual,Retail,20.00,0.4957
2,2000-01-15,National Capital region,Metropolitan Manila,Metro Manila,14.604167,120.982222,"Rice (milled, superior)",KG,actual,Wholesale,18.35,0.4548
3,2000-01-15,National Capital region,Metropolitan Manila,Metro Manila,14.604167,120.982222,"Rice (regular, milled)",KG,actual,Wholesale,16.35,0.4052
4,2000-01-15,Region III,Nueva Ecija,Palayan,15.541500,121.084800,"Rice (milled, superior)",KG,actual,Retail,19.00,0.4709
5,2000-01-15,Region III,Nueva Ecija,Palayan,15.541500,121.084800,"Rice (milled, superior)",KG,actual,Wholesale,18.00,0.4461
6,2000-01-15,Region III,Nueva Ecija,Palayan,15.541500,121.084800,"Rice (regular, milled)",KG,actual,Retail,18.10,0.4486
7,2000-01-15,Region IX,Zamboanga del Sur,Zamboanga City,6.910255,122.071715,"Rice (regular, milled)",KG,actual,Retail,16.90,0.4188
8,2000-01-15,Region VI,Iloilo,Iloilo City,10.696944,122.564444,"Rice (milled, superior)",KG,actual,Retail,20.40,0.5056
9,2000-01-15,Region VI,Iloilo,Iloilo City,10.696944,122.564444,"Rice (milled, superior)",KG,actual,Wholesale,15.60,0.3866
10,2000-01-15,Region VI,Iloilo,Iloilo City,10.696944,122.564444,"Rice (regular, milled)",KG,actual,Retail,17.80,0.4411
